In [ ]:
import pandas as pd
import numpy as np
from scipy.fft import fft, ifft
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Función para calcular el ciclo circadiano utilizando la Transformada de Fourier
def extract_circadian_cycle(data):
    fft_result = fft(data)
    n = len(data)
    freq = np.fft.fftfreq(n, d=1/24)
    
    # Filtrar para mantener solo frecuencias cercanas a 1 ciclo diario
    fft_filtered = fft_result.copy()
    fft_filtered[(freq < 0.8) | (freq > 1.2)] = 0
    
    # Reconstrucción de la señal para obtener el ciclo circadiano
    return np.real(ifft(fft_filtered))
    
def extract_circadian_cycle2(data, sample_rate=24):
    # Eliminar tendencia
    detrended = data - data.mean()

    # FFT con ventana
    window = np.hanning(len(detrended))
    fft_result = fft(detrended * window)

    # Análisis espectral
    freq = np.fft.fftfreq(len(data), d=1/sample_rate)
    power = np.abs(fft_result)**2

    # Identificar frecuencia circadiana dominante
    daily_freq_mask = (freq >= 0.8) & (freq <= 1.2)
    if not np.any(daily_freq_mask):
        print("Advertencia: No se detectó ritmo circadiano claro")

    # Filtrado adaptativo
    fft_filtered = fft_result.copy()
    fft_filtered[~daily_freq_mask] *= 0.1  # Atenuar otras frecuencias

    return np.real(ifft(fft_filtered))

# Configuración de datos de entrada
# Cargar los datos
nombres_csv = {1} #{1,2,3,4,468,479,4003,4151,4160,4173}
for nombre_csv in nombres_csv:
    df= pd.read_csv(f'Data1_actividad/{nombre_csv}_act.csv')
    df = df.copy()

    # Convertir 'Fecha' y 'Hora' a datetime e indexar el DataFrame
    df['datetime'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')
    df = df.set_index('datetime')
    df = df.sort_index()


    # Función para procesar una columna de actividad y calcular niveles de estrés
    def process_activity(df, activity_column):
        daily_cycles = {}
        
        # Extraer ciclo circadiano para cada día y almacenarlo
        for date, group in df.groupby(df.index.date):
            if len(group) == 24:  # Verificar 24 horas de datos
                daily_cycles[date] = extract_circadian_cycle(group[activity_column].values)

        # Calcular el ciclo circadiano promedio
        average_cycle = np.mean(list(daily_cycles.values()), axis=0)

        # Calcular distancias entre el ciclo promedio y los ciclos diarios
        hourly_distances = []
        for date, cycle in daily_cycles.items():
            for hour in range(24):
                hourly_distances.append({
                    'Fecha': date,
                    'Hora': hour,
                    'distancia': euclidean(average_cycle, cycle)
                })

        distance_df = pd.DataFrame(hourly_distances)

        # Calcular umbrales para clasificar niveles de estrés
        min_distance = distance_df['distancia'].min()
        max_distance = distance_df['distancia'].max()
        threshold1 = min_distance + (max_distance - min_distance) / 2
        threshold2 = threshold1 + (max_distance - threshold1) / 2
        threshold3 = max_distance
        
        # Establecer umbrales de estrés usando desviación estándar
        # mean_dist = distance_df['distancia'].mean()
        # std_dist = distance_df['distancia'].std()
        # threshold1 = mean_dist + std_dist
        # threshold2 = mean_dist + 2 * std_dist
        

        # Clasificación de niveles de estrés basada en los umbrales
        def get_stress_level(distance):
            if distance <= threshold1:
                return 'normal'
            elif distance <= threshold2:
                return 'alerta'
            else:
                return 'peligro'

        # Aplicar la función de nivel de estrés a cada distancia
        distance_df['Nivel_estres'] = distance_df['distancia'].apply(get_stress_level)
        
        return distance_df[['Fecha', 'Hora', 'Nivel_estres']]

    # Procesar ambas columnas de actividad y calcular niveles de estrés
    stress_levels_1 = process_activity(df, 'activity_level').rename(columns={'Nivel_estres': 'Nivel_estres_1'})
    stress_levels_2 = process_activity(df, 'activity_level_2').rename(columns={'Nivel_estres': 'Nivel_estres_2'})

    # Fusionar niveles de estrés con el DataFrame original
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    stress_levels_1['Fecha'] = pd.to_datetime(stress_levels_1['Fecha'])
    stress_levels_2['Fecha'] = pd.to_datetime(stress_levels_2['Fecha'])
    df['Hora'] = df['Hora'].astype(int)
    stress_levels_1['Hora'] = stress_levels_1['Hora'].astype(int)
    stress_levels_2['Hora'] = stress_levels_2['Hora'].astype(int)

    # Fusionar todos los DataFrames en df_final
    df_final = df.merge(stress_levels_1, on=['Fecha', 'Hora'], how='left')
    df_final = df_final.merge(stress_levels_2, on=['Fecha', 'Hora'], how='left')

    #df_final.to_csv(f'Datos_entrenamiento/{nombre_csv}_e.csv', index=False) 
    #df_final.to_csv(f'Datos_prueba/{nombre_csv}_p.csv', index=False)   


In [5]:
df_final

,Fecha,Hora,period eating,period other,period resting,period rumination,period eating_count,period other_count,period resting_count,period rumination_count,activity_level,activity_level_2,Nivel_estres_1,Nivel_estres_2
0,2024-01-01,0,0.079,0.0,0.410,0.511,1.0,0.0,4.0,2.0,-0.209528,-1.025107,normal,normal
1,2024-01-01,1,0.000,0.0,0.652,0.348,0.0,0.0,1.0,1.0,-0.327217,-0.443815,normal,normal
2,2024-01-01,2,0.000,0.0,0.443,0.557,0.0,0.0,1.0,1.0,-0.249102,-0.443815,normal,normal
3,2024-01-01,3,0.000,0.0,0.462,0.538,0.0,0.0,2.0,2.0,-0.256203,-0.887631,normal,normal
4,2024-01-01,4,0.000,0.0,0.711,0.289,0.0,0.0,1.0,1.0,-0.349269,-0.443815,normal,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3931,2024-06-30,19,1.000,0.0,0.000,0.000,1.0,0.0,0.0,0.0,0.261279,0.200319,alerta,normal
3932,2024-06-30,20,0.341,0.0,0.000,0.659,1.0,0.0,0.0,1.0,0.034052,-0.074599,alerta,normal
3933,2024-06-30,21,0.000,0.0,0.551,0.449,0.0,0.0,1.0,2.0,-0.289468,-0.718734,alerta,normal
3934,2024-06-30,22,0.416,0.0,0.000,0.584,1.0,0.0,0.0,1.0,0.059913,-0.074599,alerta,normal
